<a href="https://colab.research.google.com/github/kayleepho/MATH-5750-Project-4/blob/main/project4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Math 5750/6880: Mathematics of Data Science \
Project 4

# 1. Exploratory Analysis

Use the following code to download the dataset from
[https://www.kaggle.com/code/mineshjethva/eda-pulsedb/notebook](https://www.kaggle.com/code/mineshjethva/eda-pulsedb/notebook). The dataset is described in the paper [https://doi.org/10.3389/fdgth.2022.1090854](https://doi.org/10.3389/fdgth.2022.1090854).

I would recommend saving the data files to a google drive (or your local machine) so that you don't have to download them again. Note that the 5 data files correspond to the 5 columns in Table 4 of the paper.


In [1]:
# download the data from kagglehub
# The dataset is 17.3 G
# This took about 15min using university wifi and, if
# you save the data, you should only have to do it once

import kagglehub
path = kagglehub.dataset_download("weinanwangrutgers/pulsedb-balanced-training-and-testing")
print("Path to dataset files:", path)

100%|██████████| 17.3G/17.3G [07:20<00:00, 42.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/weinanwangrutgers/pulsedb-balanced-training-and-testing/versions/4


In [3]:
# run this block to move the data to a permanent directory in your drive

import os, glob
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR = "/content/drive/MyDrive/pulsedb/"
!mkdir -p $DATA_DIR
!cp -r $path/* $DATA_DIR

Mounted at /content/drive


In [4]:
# run this block after data is saved to your drive

import os, glob
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR = "/content/drive/MyDrive/pulsedb/"

mat_files = sorted(glob.glob(os.path.join(DATA_DIR, "**", "*.mat"), recursive=True))
print(f"Found {len(mat_files)} .mat files")
for f in mat_files:
    print(" -", f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 5 .mat files
 - /content/drive/MyDrive/pulsedb/VitalDB_AAMI_Cal_Subset.mat
 - /content/drive/MyDrive/pulsedb/VitalDB_AAMI_Test_Subset.mat
 - /content/drive/MyDrive/pulsedb/VitalDB_CalBased_Test_Subset.mat
 - /content/drive/MyDrive/pulsedb/VitalDB_CalFree_Test_Subset.mat
 - /content/drive/MyDrive/pulsedb/VitalDB_Train_Subset.mat


Now we'll load the data. The data is about 20GB, which exceeds the colab basic RAM allocation. You can check your RAM using

`!cat /proc/meminfo`

You should upgrade to colab pro, which is free for students.

[https://colab.research.google.com/signup](https://colab.research.google.com/signup)

Then in 'change runtime type' click A100 GPU and high RAM.

In [1]:
# load the data
# the subject information is stored in a pandas df
# the Signals (ECG, PPG, ABP) are stored in numpy arrays
# this block takes 11 minutes to execute

!cat /proc/meminfo

!pip install mat73
import mat73
import pandas as pd
import numpy as np

def load_mat_file(file_path):
    data_dict = mat73.loadmat(file_path)['Subset']
    print('finished loading'+file_path)
    # print(data_dict.keys())

    # first handle Signals
    ECG = data_dict['Signals'][:,0,:]
    PPG = data_dict['Signals'][:,1,:]
    ABP = data_dict['Signals'][:,2,:]
    data_dict.pop("Signals", None)

    data_dict['Age'] = data_dict['Age'].tolist()
    data_dict['BMI'] = data_dict['BMI'].tolist()
    data_dict['DBP'] = data_dict['DBP'].tolist()
    data_dict['Gender'] = [1 if x[0] == 'M' else 0 for x in data_dict['Gender']]
    data_dict['Height'] = data_dict['Height'].tolist()
    data_dict['SBP'] = data_dict['SBP'].tolist()
    data_dict['Subject'] = [x[0] for x in data_dict['Subject']]
    data_dict['Weight'] = data_dict['Weight'].tolist()

    data_df = pd.DataFrame(data_dict)
    print('constructed df')

    return data_df, ECG, PPG, ABP

df_CalBased_Test, ECG_CalBased_Test, PPG_CalBased_Test, ABP_CalBased_Test = load_mat_file(DATA_DIR+'VitalDB_CalBased_Test_Subset.mat')
df_Train, ECG_Train, PPG_Train, ABP_Train = load_mat_file(DATA_DIR+'VitalDB_Train_Subset.mat')

MemTotal:       13286956 kB
MemFree:        11500484 kB
MemAvailable:   12131180 kB
Buffers:           59756 kB
Cached:           626640 kB
SwapCached:            0 kB
Active:           363808 kB
Inactive:         920588 kB
Active(anon):       1604 kB
Inactive(anon):   598508 kB
Active(file):     362204 kB
Inactive(file):   322080 kB
Unevictable:          20 kB
Mlocked:              20 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               992 kB
Writeback:             0 kB
AnonPages:        598012 kB
Mapped:           246328 kB
Shmem:              2100 kB
KReclaimable:     272236 kB
Slab:             328568 kB
SReclaimable:     272236 kB
SUnreclaim:        56332 kB
KernelStack:        7080 kB
PageTables:         9592 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6643476 kB
Committed_AS:    3440832 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       95924 kB
VmallocChunk:    

NameError: name 'DATA_DIR' is not defined

In [ ]:
# df_CalBased_Test has 51720 entries
print(df_CalBased_Test.keys())
print(df_CalBased_Test.info())
print(df_CalBased_Test.describe())
df_CalBased_Test

In [ ]:
# 1293 subjects, 40 samples/ subject = 51720 samples
df_CalBased_Test['Subject'].value_counts()

The PulseDB dataset is a large, cleaned dataset based on MIMIC-III
and VitalDB for benchmarking cuff-less blood pressure estimation methods1.
- Read the paper to understand the dataset. The full dataset is available here: https://github.com/pulselabteam/PulseDB
and a subset is here: https://www.kaggle.com/code/mineshjethva/eda-pulsedb
- Use the provided code to import the PulseDB dataset.
- Conduct an exploratory analysis.

In [ ]:
# your code here

#2. Blood Pressure Prediction

Using the PulseDB dataset, we will build models to predict
arterial blood pressure (ABP) from raw electrocardiogram (ECG) and photoplethysmogram (PPG) signals.
- First, develop regression models that take ECG and PPG siganls and predict diastolic blood pressure (DBP) and systolic blood pressure (SBP).
- First try a linear model and then try models of increasing complexity, e.g., fully connected NN, RNN, LSTM, transformer etc. Use the provided train/test data split.
- Challenge problem: Develop models for the sequence-to-sequence prediction problem of predicting the full ABP waveform from ECG and PPG signals.
- Again, first try a linear model to establish a baseline and then increase complexity by considering, e.g., fully connected NN, RNN,
LSTM, transformer etc


In [ ]:
# your code here

#3. Generative Modeling

Using the PulseDB dataset, we will develop generative models for
arterial blood pressure (ABP) signals.
- First, perform a Principle Component Analysis (PCA) to understand the dimensionality of the ABP signals. Perform an “elbow analysis” to determine the intrinsic linear dimension of the data.
- Then, train a fully connected or 1D convolutional autoencoder (AE) to minimize reconstruction loss (MSE between reconstructed and true ABP). Again, perform an elbow analysis to choose a latent dimension. - Compare this dimension to the dimension obtained via the PCA analysis. Use the decoder to generate some new ABP signals. Include plots with examples of both data and generated signals in your report.

In [ ]:
# your code here